In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
# Get optimal hyperparameters for classifier
df = pd.read_json("../reports/tune_clf.jsonl", orient='records', lines=True)
hparams = ['dataset', 'lr', 'n_hidden_layers', "n_hidden_units"]
df['accuracy'] = df.apply(lambda row: (np.array(row['preds']) == np.array(row['labels'])).mean(), axis=1)
idcs_best = df.groupby('dataset')['accuracy'].idxmax()
df.loc[idcs_best][hparams + ['accuracy']]

,dataset,lr,n_hidden_layers,n_hidden_units,accuracy
95,Covertype,0.03125,1,1024,0.7096
238,Insects abrupt,0.06250,1,1024,0.9726
203,Rotated MNIST,0.03125,1,1024,0.9192


In [108]:
# Get optimal hyperparameters for autoencoder
df = pd.read_json("../reports/ae_tune_online.jsonl", orient="records", lines=True)
hparams = [
    "dataset",
    "online_finetuning",
    "epochs",
    "lr",
    "lr_online",
    "n_hidden_layers",
    "n_hidden_units",
]

df["anom_scores"] = df["anom_scores"].apply(np.array)
df["is_anom"] = df["is_anom"].apply(np.array)

df["roc_auc"] = df.apply(
    lambda row: roc_auc_score(row["is_anom"], row["anom_scores"]), axis=1
)

idcs_max = df.groupby(["dataset", "online_finetuning", "epochs"])["roc_auc"].idxmax()
df.loc[idcs_max][hparams + ["roc_auc"]]

,dataset,online_finetuning,epochs,lr,lr_online,n_hidden_layers,n_hidden_units,roc_auc
578,Covertype,False,8,1.00,1.00,1,64,1.000000
524,Covertype,True,0,0.25,1.00,2,256,0.924711
2,Covertype,True,8,1.00,0.50,1,64,1.000000
708,Insects abrupt,False,8,0.25,1.00,2,512,0.833288
540,Insects abrupt,True,0,0.25,1.00,1,512,0.671406
852,Insects abrupt,True,8,0.25,0.25,2,512,0.828001
577,Rotated MNIST,False,8,1.00,1.00,1,64,0.716890
433,Rotated MNIST,True,0,0.25,1.00,1,64,0.443077
721,Rotated MNIST,True,8,1.00,0.25,1,64,0.739951
